## 14. Индексные дескрипторы файлов файловой системы ext. Линковка (жесткая и мягкая)

Благодаря своему родству с файловыми системами UFS, многие файловые системы используют общую описательную терминологию. Реализации базовых объектов часто изменяются, но термины по-прежнему широко употребляются системными администраторами как обозначения фундаментальных понятий. Например, индексные дескрипторы (“inodes”) — это записи фиксированной длины в таблице, каждый элемент которой хранит информацию об отдельном файле. Ранее они заносились в эту таблицу в момент создания файловой системы, но в настоящее время некоторые файловые системы создают их динамически при необходимости. В любом случае индексный дескриптор обычно имеет идентификационный номер, который можно увидеть с помощью команды `ls -i`.

Индексные дескрипторы указывают на элементы каталога. При создании жесткой ссылки на существующий файл создается новый элемент каталога, но не новый индексный дескриптор.
В системах с заранее размещенными индексными дескрипторами необходимо заранее решить, сколько их следует создать. Поскольку невозможно предсказать, сколько их понадобится в будущем, команды создания файловой системы используют эмпирическую формулу, основанную на объеме тома и среднем размере файлов. Если вы планируете хранить огромное количество маленьких файлов, это число следует увеличить.

Суперблок (superblock) — это запись, описывающая характеристики файловой системы. Она содержит информацию о длине блока диска, размере и местоположении таблиц индексных дескрипторов, карту блоков и информацию о их использовании, размер групп блоков и некоторые другие важные параметры файловой системы. Поскольку повреждение суперблока может привести к потере очень важной информации, его несколько копий хранятся в разных местах.
Для повышения производительности файловые системы кешируют блоки диска.

Кешировать можно все блоки диска, включая суперблоки, блоки индексных дескрипторов и информацию о каталоге. Кеши обычно не допускают “сквозной записи” (“writethrough”), поэтому может возникнуть задержка между моментом, когда приложение считает, что блок записан, и моментом, в который блок действительно записывается на диск. Приложения могут требовать от файла более предсказуемого поведения, но в этом случае пропускная способность снизится.

Системный вызов `sync` направляет модифицированные блоки в место их постоянного хранения на диске, стараясь моментально обеспечить полную согласованность дисковой файловой системы. Это периодическое сохранение минимизирует потерю данных, которая может произойти при сбое из-за многочисленных несохраненных блоков.

Файловые системы могут самостоятельно выполнять синхронизацию своего расписания или предоставить это операционной системе. Современные файловые системы имеют механизм журналирования, минимизирующий или исключающий возможность повреждения их структуры при сбое, поэтому частота синхронизации должна зависеть от того, сколько блоков могут оказаться потерянными при сбое.

В файловой системе карта дисковых блоков — это таблица содержащихся в ней свободных блоков. При записи новых файлов система изучает эту карту, чтобы найти эффективную схему хранения. Записи об использовании блоков содержат важную информацию об уже используемых блоках. В файловых системах, поддерживающих логические разделы, эта информация может быть значительно сложнее, чем обычная битовая карта, которая применялась в старых файловых системах.

1. Именное обозначение (filename)
2. Индексный дескриптор/метаданные
3. Блоки данных/данные (всегда присутствует минимум один файловый дескриптор (i-node), если его нет, блок будет перезаписан)

```
block1 \
        inode
block2 /     \
              \
               Data
              /
block3 - inode   
```

>0,3%-0,7% от общего объема жд - inode

Ext не умеют динамически увеличивать количество нод.
В случае исчерпания всех индексных дескрипторов возможности создания файла/директории/другую структуру не получится.
Индексные дескрипторы привязаны к области, которая размечена в фс и в ее рамках они уникальны. Если в системе имеется несколько жд (или один жд разбит на несколько разделов), каждая из ext может содержать ноды с одинаковыми номерами.

Из данных суперблока можно извлечь следующую информацию:
- общее число блоков и нод в фс
- размер блока в фс
- число свободных нод или блоков в текущей фс
- размер индексного дескриптора
- идентификатор фс
- дата последней проверки фс
- количество произведенных монтирований
- форк состояния фс

Специальные ссылки “.” и “..” обозначают сам каталог и его родительский каталог соответственно. Такие ссылки нельзя удалить. Поскольку корневой каталог находится на вершине иерархии, ссылка “..” в нем эквивалентна ссылке “.”

Имя файла в действительности хранится в родительском каталоге, а не в самом файле. На файл можно ссылаться из нескольких каталогов одновременно и даже из нескольких элементов одного и того же каталога, причем у всех ссылок могут быть разные имена. Это создает иллюзию того, что файл одновременно присутствует в разных каталогах. При жесткой линковке файлов дополнительные дескрипторы не создаются

Эти дополнительные **“жесткие” (фиксированные) ссылки** (которые следует отличать от символических, или “мягких”) можно считать синонимами для исходных файлов, и с точки зрения файловой системы все ссылки на файл эквивалентны. Файловая система подсчитывает количество ссылок на каждый файл и при удалении файла не освобождает блоки данных до тех пор, пока не будет удалена последняя ссылка на него. **Ссылки не могут указывать на файл, находящийся в другой файловой системе.**

Жесткие ссылки создаются командой `ln` и удаляются командой `rm`. Синтаксис команды `ln` легко запомнить, поскольку она является “зеркальным отражением” команды `cp`. Команда `cp oldfile newfile` создает копию файла `oldfile` с именем `newfile`, а команда `ln newfile oldfile` преобразует имя `newfile` в дополнительную ссылку на файл `oldfile`.

Для того чтобы узнать, сколько ссылок существует на данный файл, используйте команду `ls -l`

**Символическая, или “мягкая”, ссылка**позволяет вместо имени файла указывать его псевдоним. Столкнувшись при поиске файла с символической ссылкой, ядро извлекает хранящееся в ней имя. Разница между жесткими и символическими ссылками состоит в том, что жесткая ссылка является прямой, т.е. указывает непосредственно на индексный дескриптор файла, тогда как символическая ссылка указывает на файл по имени. Файл, адресуемый символической ссылкой, и сама ссылка представляют собой разные объекты файловой системы.

Символические ссылки создаются командой `ln -s` и удаляются командой `rm`. Они могут содержать произвольное имя, т.е. разрешается указывать на файлы, хранящиеся в других файловых системах, и даже на несуществующие файлы. Иногда несколько символических ссылок образуют петлю.


## 15. Отличие ext2, ext3 и ext4. Журналирование. 

- семейство ext: ext2/ext3/ext4

ext2 — нежурналируемая файловая система. Ограничения: размер файла 2 Тб, размер тома 32 Тб. Неэффективная работа с директориями с большим количеством файлов.

Ext3 — журналируемая ФС с поддержкой htree для более эффективной работы с директориями, содержащими большое кол-во файлов.

ext4 — дальнейшее развитие. Ограничение размера ФС — 1 экзобайт, размера файла — 32 Тб. Поддержка преаллокации, отложенной аллокации, [extent-ов](https://en.wikipedia.org/wiki/Extent_(file_systems)).

Файловая система ext3 расширяла возможности журналирования (journaling), которые имелись в существовавшем коде ext2, и представляла собой концептуально простую модификацию, резко повышавшую надежность. Еще более интересным является то,
что расширения в файловой системе ext3 были реализованы без изменения фундаментальной структуры файловой системы ext2. Фактически **вы можете монтировать файловую систему ext3 так, будто это файловая система ext2**, просто без дополнительных возможностей журналирования.

Файловая система ext3 резервирует область памяти на диске для журнала. Журнал размещается на диске так, будто он представляет собой обычный файл в корневом разделе файловой системы и не является отдельным структурным компонентом.

При выполнении операции с файловой системой требуемые модификации сначала записываются в журнал. Когда обновление журнала завершено, в него вносится “запись о фиксации” (“commit record”), чтобы отметить конец вводимых данных. Только затем происходит модификация обычной файловой системы. Если в процессе обновления происходит сбой, файловая система использует журнал для реконструкции полностью согласованной файловой системы. После сбоя аппаратного обеспечения определенного типа состояние файловой системы ext3 можно практически мгновенно восстановить.

Файловая система ext4 представляет собой определенное улучшение своих предшественников. Она расширяет пределы максимально допустимой памяти, увеличивает производительность некоторых операций и позволяет использовать для выделения памяти не отдельные блоки диска, а “логические диапазоны” (“extents”). Дисковый формат вполне совместим с файловыми системами ext2 и ext3 и допускает их монтирование в виде файловой системы ext4. Более того, если не используются логические диапазоны, то **файловые системы ext4 могут монтироваться так, будто они являются файловыми системами ext3**.

- xfs — журналируемая ФС с акцентом на параллелизм и поддержку эффективной работы с большими файлами. Ограничение размера — 8 экзобайт.

- btrfs позиционируется как ФС нового поколения. Поддерживает copy on write, прозрачное сжатие, снепшоты, объединение устройств в raid-массивы. Максимальный размер — 8 экзобайт.

- множество файловых систем для использования на определенных типах устройств, разработанных для специальных задач и т. п. (f2fs, overlayfs, тысячи их).

ext3 не нуждается в дефрагментации ext4 в два раза больше и эффективнее, поскольку является логичным эволюционным витком, ведь именно с ней происходит переход на 64-битные фс

## 16. Права доступа к файлам и папкам ОС Linux. Липкие биты и их функционал.

В традиционной модели файловой системы UNIX и Linux каждому файлу соответствует набор из девяти битов режима. Они определяют, какие пользователи имеют право читать файл, записывать в него данные или запускать его на выполнение. Вместе с другими тремя битами, которые в основном влияют на работу исполняемых файлов, этот набор образует код, или режим, доступа к файлу.

Двенадцать битов режима хранятся вместе с четырьмя дополнительными битами, определяющими тип файла. Эти четыре бита устанавливаются при создании файла и не подлежат изменению. Биты режима могут изменяться владельцем файла или суперпользователем с помощью команды chmod (“change mode” — изменить режим). Просмотр значений этих битов осуществляется с помощью команды `ls -l` (или `ls -ld` в случае каталога).

Код доступа удобно записывать в виде восьмеричного числа, так как каждая цифра в нем представляется тремя битами. Три старших бита (в коде доступа им соответствуют восьмеричные значения 400, 200 и 100) служат для управления доступом к файлу его владельца. Вторые три бита (40, 20 и 10) задают доступ для членов группы. Последние три бита (4, 2 и 1) определяют доступ к файлу остальных пользователей. Старший бит каждой триады — это бит чтения, средний — бит записи, младший — бит выполнения.

Будучи установленным для каталога, бит выполнения (в этом контексте его часто называют битом поиска) разрешает переходить в каталог или проходить через него при доступе к файлу, но без получения списка содержимого каталога. 

Система не работает на прямую с идентификатором пользователя в буквенном формате, он идентифицирует его числом (UID /etc/passwd). Аналогичная ситуация происходит с группой. Для ее обозначения используется групповой идентификатор GID (/etc/group). У каждого пользователя в системе есть группа. Группа, которая используется по умолчанию в виде идентификатора прописана сразу за идентификатором пользователя. Группа в конфиге может быть создана пустой (без пользователей). 

### Sticky bit

Биты, которым в коде доступа соответствуют восьмеричные значения 4000 и 2000, — это биты смены идентификатора пользователя (setuid) и идентификатора группы (setgid). Если эти биты установлены для исполняемых файлов, они позволяют программам получать доступ к файлам и процессам, которые при прочих обстоятельствах недоступны пользователю, выполняющему эти программы. 

Если бит setgid установлен для каталога, то создаваемые в нем файлы будут принимать идентификатор группы каталога, а не группы, в которую входит владелец файла. Это упрощает пользователям, принадлежащим к одной группе, совместный доступ к каталогам.

Бит, которому в коде доступа соответствует восьмеричное значение 1000, называется дополнительным **(sticky bit)**. В первых UNIX-системах дополнительный бит запрещал выгрузку программ из памяти. Сегодня он утратил свое значение и, попросту, игнорируется.

Если дополнительный бит установлен для каталога, то файловая система позволит удалять и переименовывать его файлы только владельцу каталога, владельцу файла или суперпользователю. Иметь одно лишь право записи в каталог недостаточно. Такая мера позволяет несколько лучше защитить каталоги вроде `/tmp`

`*sticky uid - SUID* 100 [000 000 000]`

`*SGID* 010 [000 000 000]`

Восьмиричный sticky-bit 1000 +t может возвести только root. Удаление файлов из такого каталога возможно только владельцем файла или рутом. Остальные будут лишены права удалить файл, даже если их группа имеет доступ. Владелец каталога сможет убрать бит, но возвести обратно не сможет.

Пользователь www id 33

Код доступа к файлу можно изменить с помощью команды `chmod`. Такое право есть лишь у владельца файла и пользователя root. В первых UNIX-системах код доступа задавался в виде восьмеричного числа. В современных версиях поддерживается также система мнемонических обозначений. При использовании мнемонического синтаксиса вы объединяете множество исполнителей (u — пользователь, g — группа или о — другой) с оператором (`+` — добавить, `-` — удалить и `=` — присвоить) и набором прав доступа

![](mnem.png)

## 17. Подход используемый в ОС Linux для взаимодействия пользователя и ядра.

`modprobe` - Добавить модуль ядра

**Пользовательское пространство** — адресное пространство виртуальной памяти операционной системы, отводимое для пользовательских программ, в отличие от пространства ядра, которое резервируется для работы ядра операционной системы, его расширений и, возможно, некоторых драйверов устройств.

Каждый процесс в пользовательском пространстве обычно выполняется в собственной области виртуальной памяти, и при отсутствии явной необходимости, не может получить доступа к памяти, используемой другими процессами. Такой подход является базисным для обеспечения защиты памяти большинства современных операционных систем, и своего рода «фундаментом» для обеспечения права доступа. В зависимости от привилегий процесс может запросить ядро отобразить часть адресного пространства другого процесса на своё, как, например, это делают отладчики. Программы также могут запрашивать для себя область разделяемой памяти совместно с другими процессами.

![](userspace.png)

**Процесс** — это абстракция, используемая в операционных системах UNIX и Linux для описания выполняющейся программы. Процесс представляет собой системный объект, посредством которого можно контролировать обращения программы к памяти, центральному процессору и ресурсам ввода-вывода.

Концепция, согласно которой как можно больше работы должно выполняться в контексте процессов, а не в ядре, является частью философии UNIX. Системные и пользовательские процессы подчиняются одним и тем же правилам, благодаря чему управление ими осуществляется с помощью единого набора команд.

Приложение может обращаться к ядру, используя следующие механизмы:

- [системные вызовы](https://blog.packagecloud.io/eng/2016/04/05/the-definitive-guide-to-linux-system-calls/) - `man syscalls`

- псевдо-ФС /proc и /sys предлагают интерфейс для получения информации о состоянии различных процессов и подсистем, а также для управления ими.

Ядро по своей инициативе может управлять исполнением процесса в рамках работы планировщика, а также отправлять процессу сигналы.

Процесс состоит из адресного пространства и набора структур данных, содержащихся внутри ядра. Адресное пространство представляет собой совокупность страниц памяти, которые были выделены ядром для выполнения процесса. В него загружается код процесса и используемые им библиотеки функций, а также переменные, содержимое стеков и различная вспомогательная информация, необходимая ядру во время работы процесса. Поскольку в системах UNIX и Linux поддерживается концепция виртуальной памяти, страницы адресного пространства процесса в конкретный момент времени могут находиться либо в физической памяти, либо в разделе подкачки, т.е. на диске.
В структурах данных ядра хранится всевозможная информация о каждом процессе.

К наиболее важным относятся следующие сведения:
- таблица распределения памяти;
- текущий статус (неактивен, приостановлен, выполняется и т.п.);
- приоритет;
- информация об используемых ресурсах;
- информация о файлах и сетевых портах, открытых процессом;
- маска сигналов (запись о том, какие сигналы блокируются);
- имя владельца процесса.

## 18. Классификация сигналов. Утилита kill

**Сигналы** — это запросы на прерывание, реализуемые на уровне процессов. Определено свыше тридцати различных сигналов, и они находят самое разное применение.

- Сигналы могут посылаться между процессами как средство коммуникации.
- Сигналы могут посылаться драйвером терминала для уничтожения или приостановки процессов, когда пользователь нажимает специальные комбинации клавиш, такие как <Ctrl+C> или <Ctrl+Z>.
- Сигналы могут посылаться в самых разных целях пользователем или администратором с помощью команды `kill`.
- Сигналы могут посылаться ядром, когда процесс выполняет нелегальную инструкцию, например деление на нуль.
- Сигналы могут посылаться ядром для уведомления процесса о “представляющем интерес” событии, таком как прекращение дочернего процесса или доступность данных в канале ввода-вывода.

Два основных вида сигналов — **обрабатываемые и необрабатываемые**. К последним относятся **SIGSTOP** и **SIGKILL**, все остальные сигналы могут быть обработаны.

Когда поступает сигнал, возможен один из двух вариантов развития событий. Если процесс назначил сигналу подпрограмму обработки, то после вызова ей предоставляется информация о контексте, в котором был сгенерирован сигнал. В противном случае ядро выполняет от имени процесса действия, заданные по умолчанию. Эти действия зависят от сигнала. Многие сигналы приводят к завершению процесса, а в некоторых случаях при этом еще и создается дамп памяти.

Процедура вызова обработчика называется **перехватом сигнала**. Когда выполнение обработчика завершается, процесс возобновляется с той точки, где был получен сигнал.
Для того чтобы определенные сигналы не поступали в программу, нужно задать их игнорирование или блокирование. Игнорируемый сигнал просто пропускается и не влияет на работу процесса. Блокируемый сигнал ставится в очередь на обработку, но ядро не требует от процесса никаких действий до явного разблокирования сигнала. Обработчик
вызывается для разблокированного сигнала только один раз, даже если в течение периода блокировки поступило несколько аналогичных сигналов.

![](kill.png)

Еще один классифицирующий признак — действие по умолчанию при отсутствии явно заданного обработчика. Получение одних сигналов (например, SIGHUP, SIGINT) в отсутствие обработчика приводит к завершению процесса, получение других (например, SIGCHLD, SIGUSR1) по умолчанию игнорируется.

Сигналы BUS и SEGV также посылаются при возникновении ошибок. Сами по себе эти сигналы не имеют диагностической
ценности. Они лишь указывают на факт неправильного обращения к памяти.

Сигналы KILL и STOP нельзя ни перехватить, ни заблокировать, ни проигнорировать. Сигнал KILL приводит к уничтожению процесса, которому он посылается, а сигнал STOP приостанавливает выполнение процесса до получения сигнала CONT. Сигнал
CONT можно перехватить и проигнорировать, но нельзя заблокировать.

Утилита kill (а также одноименная встроенная команда bash) позволяет отправлять сигналы процессам, например:

- `kill -9 1234` — отправляет процессу с pid 1234 SIGKILL

- `kill -KILL 1234` — идентично предыдущей команде

- `kill -l` — выводит список сигналов.

Вся структура процессов имеет древовидную основу. Исключения: процесс-сирота (orphaned), процесс-зомби.
Все процессы в рамках одной ОС имеют разные PID.

Контекст сигнальных вызовов signal context

- SIGKILL - смерть сродителя
- SIGTERM - смерть потомков

Процесс может быть не завершен, а остановлен. Посмотреть - `jobs`. `fg` запускает джоб с плюсиком, иначе нужно писать номер

## 19. Задача об обедающих философах. Проблемы многозадачности.

- [Задача об обедающих философах](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BE%D0%B1_%D0%BE%D0%B1%D0%B5%D0%B4%D0%B0%D1%8E%D1%89%D0%B8%D1%85_%D1%84%D0%B8%D0%BB%D0%BE%D1%81%D0%BE%D1%84%D0%B0%D1%85)
- [Round-robin](https://ru.wikipedia.org/wiki/Round-robin_(%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC))

Поток выполнения, обычно именуемый просто потоком, представляет собой результат разветвления в выполнении процесса. Поток наследует многие атрибуты своего процесса (например, адресное пространство процесса), причем в рамках одного процесса могут выполняться одновременно (параллельно) сразу несколько потоков — такая модель выполнения получила название многопоточности (multithreading).

В старых однопроцессорных системах параллельное выполнение моделируется (точнее, имитируется) ядром, но в мультиядерных и многопроцессорных архитектурах потоки могут выполняться одновременно в различных ядрах. Такие многопоточные приложения, как BIND и Apache, извлекают максимальную пользу из мультиядерных систем,
поскольку эти приложения могут отрабатывать несколько запросов одновременно.

- однозадачные ОС передают управление одному-единственному процессу до момента его завершения.

- кооперативная многозадачность позволяет ОС поддерживать запуск нескольких процессов одновременно, однако, для обеспечения многозадачности необходимо, чтобы процессы передавали управление по своей инициативе. Если один из процессов, например, войдет в вечный цикл, функционирование ОС будет нарушено. См. https://en.wikipedia.org/wiki/Cooperative_multitasking

- ОС, реализующие вытесняющую многозадачность, могут прерывать выполнение процесса по своей инициативе, чтобы передавать управление другим процессам. См. https://en.wikipedia.org/wiki/Preemption_(computing)#Preemptive_multitasking

Мьютекс, семафоры, почтовые ячейки (mail boxes)

## 20. Понятие процесса, его персональный идентификатор (PID). Управления процессами

Процесс - это:

- программа на стадии выполнения
- "объект", которому выделено процессорное время
- асинхронная работа 

Ядро назначает каждому процессу уникальный идентификатор **PID**. Большинство команд и системных вызовов, работающих с процессами, требует указания конкретного идентификатора, чтобы был ясен контекст операции. Идентификаторы присваиваются по порядку по мере создания процессов

Ни в UNIX, ни в Linux нет системного вызова, который бы инициировал новый процесс для выполнения конкретной программы. Для того чтобы породить новый процесс, существующий процесс должен клонировать сам себя. Клон может заменить выполняемую программу другой.

В операции клонирования исходный процесс называют родительским, а его клон — дочерним. Помимо собственного идентификатора, каждый дочерний процесс имеет атрибут **PPID (Parent Process ID)**, который совпадает с идентификатором породившего его родительского процесса

Процессы состоят из следующих компонентов:

- Образ исполняемого кода;

- Адресное пространство - https://en.wikipedia.org/wiki/Virtual_memory

- Набор дескрипторов ввода-вывода.

- Атрибуты доступа (uid, gid, лимиты, etc).

- Контекст процессора.

См. также https://www.thegeekstuff.com/2012/03/linux-processes-memory-layout/

## 21. Приоритеты процессов (относительный и абсолютный). nice/renice

Фактор “уступчивости” — это число, по которому ядро определяет свою политику в отношении процессов, конкурирующих за право доступа к центральному процессору. Чтобы посмотреть идентификатор и значение nice нужного процесса в системе, можно воспользоваться командой `ps axl`.

Чем выше фактор уступчивости, тем ниже приоритет процесса и наоборот, отсюда и название термина. Низкое или отрицательное значение означает использование высокого приоритета: процесс ведет себя не слишком уступчиво.

Диапазон допустимых значений фактора уступчивости зависит от используемой системы, и обычно он лежит в пределах от -20 до +19. В некоторых системах используется диапазон такого же размера, но со смещением в область неотрицательных чисел (как правило, от 0 до 39). 
Несмотря на числовые различия, все системы обрабатывают значения фактора уступчивости практически одинаково. Если пользователь не предпринимает специальных мер, **дочерний процесс наследует приоритет своего родительского процесса**.

**Владелец процесса может увеличить фактор уступчивости, но не может уменьшить его**, даже чтобы вернуться к стандартному значению. Это не позволяет процессам с низким приоритетом порождать высокоприоритетных потомков. Суперпользователь может устанавливать произвольные значения фактора уступчивости.

К сожалению, уровень производительности подсистемы ввода-вывода растет не так стремительно, как быстродействие центральных процессоров, поэтому жесткие диски стали основным узким местом в большинстве операционных систем. Фактор уступчивости никак не влияет на подсистемы управления памятью и вводом-выводом, поэтому даже низкоприоритетный процесс способен монополизировать эти ресурсы или захватить непропорционально большую их часть.

Фактор уступчивости можно установить при создании процесса. Это делается с помощью команды `nice`. Команда `renice` позволяет изменять приоритет выполняемого процесса. Первая из этих команд принимает в качестве аргумента строку запуска процесса, а вторая — идентификатор процесса либо имя пользователя.

При абсолютной приоретизации процесс с более высоким приоритетом не может быть вытеснен ради процесса с более низким приоритетом; при отностиельной приоретизации процессы получают доли процессорного времени, соответствующие приоритетам. Механизм выставления приоритетов с помощью nice в большинстве Unix-сисием (FreeBSD, Mac OS) использует абсолютную приоретизацию; однако в Linux он позволяет задавать относительные приоритеты.

https://developer.ibm.com/tutorials/l-lpic1-103-6/

Для задания абсолютных приоритетов используются планировщики реального времени.

См. http://man7.org/linux/man-pages/man7/sched.7.html — раздел «Scheduling policies». 

## 22. Системный вызов fork()

Не используется уже как минимум лет 20, в отличие от [одноименной библиотечной функции](https://github.com/lattera/glibc/blob/master/sysdeps/unix/sysv/linux/arch-fork.h) Для порождения новых процессов используется системный вызов [clone()](http://man7.org/linux/man-pages/man2/clone.2.html). В зависимости от передаваемых флагов, он может как породить новый поток, так и новый процесс. Разница между потоками и процессами в линуксе минимальна и заключается в том, что **все потоки одного процесса работают в одном адресном пространстве, а у каждого процесса адресное пространство свое**.

При порождении дочернего процесса вместо копирования памяти родителя используется подход copy on write, при котором фактическое выделение отдельных страниц памяти для потомка происходит лишь по мере изменения их содержимого в родительском или дочернем процессе.

Система может отреагировать на fork тремя способами:
    
- `fork < 0` - возвращает родителю код ошибки
- `fork > 0` - возвращает PID дочернего процесса в случае успеха
- `fork = 0` - особая ситуация, говорящая о том, что вы являетесь дочерним процессом, созданным сразу за получением результата этой операции
    
Процесс-потомок, поскольку содержит копию основного процесса, имеет индексные дескрипторы файлов, сокеты и другие устр-ва, к которым обращался процесс-родитель

## 23. Переменные окружения пользователя. Как их задать/прочитать/изменить?

Переменные окружения присваиваются не пользователю, а конкретному процессу и наследуются при порождении новых процессов.

Переменные окружения конкретного процесса доступны в `/proc/<pid>/environ`. В качестве разделителя используется не перенос строки (поскольку в значении каждой переменной могйт быть переносы строки), а **нулевой символ**, поэтому для просмотра удобно использовать утилиту `strings`. Пример:

`sudo strings /proc/1/environ`

При создании пользовательской сессии есть несколько источников переменных окружения. Во-первых, применяются переменные, заданные в `/etc/environment` (использование не рекомендуется, т. к. заданные переменные окружения могут влиять на работу всех демнов, например, cron). Затем применяются стартовые скрипты для командной оболочки (для bash - `/etc/bash.bashrc`, `~/..bashrc`); во многих случаях там задаются переменные окружения, необходимые для работы пользовательской сессии: PATH, LC_*, PS1 и т. п.

Получение списка переменных окружения — `printenv` Задание переменное окружения: `export NAME=VALUE`

Получение значения конкретной переменной: `echo $NAME`

## 24. Формат ELF. Типы ELF файлов.

[Подробное описание](https://en.wikipedia.org/wiki/Executable_and_Linkable_Format)

ELF (Executable and linkable format) — формат:

- исполняемых файлов;

- объектных файлов (объектный файл — результат работы компилятора; исполняемый файл формируется из объектных посредством [линковки](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BC%D0%BF%D0%BE%D0%BD%D0%BE%D0%B2%D1%89%D0%B8%D0%BA));

- динамически загружаемых библиотек (shared libraries);

- core-дампов (дампов памяти процесса, обычно формируемых для обработки отладчиком).

Он бывает трех типов - перемещаемый, разделяемый и исполняемый:


```
|-------|
|0xfff..|
|-------|
| steak |
|-------|
| heap  |
|-------|
| data  |
|-------|
| code  |
|-------|
|0x000..|
|-------|

virtual memory
```
В момент запуска ПО вся информация, которая должна быть выполнена, загружается в сектор `code`
Любой процесс, исполняемый в рамках системы перед своим запуском проходит два этапа:

1. **Fork**

    Поскольку любое ПО в рамках ОС считает, что исполняется только оно, ему предоставляется сегмент памяти, для записи своих данных, кода и изменяемых значений (*heap* и *stack*). Место, в которое эти данные будут записаны, принято называть виртуальной памяти. ВП сильно разряжена (имеет большое кол-во незаполненных ячеек (нули) и соотносится с реальной памятью по правилам, которые мы рассмотрим далее). Создание нового процесса начинается с того, что от процесса родителя поступает системный вызов (*syscall*) с просьбой для ОС создать его дубликат. 

2. **Exec**

    После того, как родительский экземпляр программы породил дочерний, дочерний экземпляр может загрузить себя на исполнение любой исполняемый ELF (он бывает трех типов - перемещаемый, разделяемый и исполняемый)
    
    - Перемещаемые файлы. Хранит инструкции и данные, которые могут быть связанными с разделяемыми файловыми объектами или исполняемыми
    - Разделяемый объектный файл. Так же как и перемещаемый файл, содержит инструкцию, которые могут быть связаны с другими перемещаемыми файлами или разделяемыми объектными файлами, в результате чего создается новый объектный файл.
    - Исполняемый объектный файл. Содержит полный объект, позволяющий создать процесс.
    
   Исполняемый файл должен иметь в себе заголовок (elf-header), одинаковый для 32 (dword-значения) и 64 битных систем. Он:
   
   - Содержит основные хар-ки: тип, версия формата, архитектура процессора, адрес точки входа, размеры и смещения остальных частей файла
   - Всегда распологается в начале файла
   - Размер 52 байта для 32-х разрядных и 64 байта для 64-х разрядных. Несмотря на то, что формат для обоих хедеров идентичный, последний получается больше из-за наличия внутри более длинных ссылок (8 байт против 4)
   
    Таблица заголовков содержит в себе таблицы, каждая из которых описывает отдельный сегмент программы, его атрибуты, используемые операционной системой для подготовки программы к исполнению. В отличии от хедера, таблица может распологаться в любой части файла. Смещение относительно начала указывается в заголовке. Третья часть - в заголовке секции. Опциональный параметр, elf-файл будет загружен, даже если заголовки секций не будут найдены. Позволяют оптимально разместить программу компоновщику. 
    
**Shared memory** - сегмент памяти, информация из которого предоставляется одновременно сразу нескольким процессам (там находятся динамически-линкуемые библиотеки)  

В процессе выполнения `exec` выполняется реаллокация (отвязывание) shared-memory сегмента. После чего из elf-хедера выполняется чтение сегментов данных кода и т.д., которые переносятся в соответствующие сегменты процесса-потомка, а данные, оставшиеся там от форка родителя, удаляются. Стек и хип приводятся в первоначальный вид, после этого из шаред-мемори выполняется аллокация (присоединение) разделяемых блоков памяти. Если в шеред-мемори отсутствуют необходимые разделяемые эльф-объекты. По заключению подготовки виртуальной памяти, для процессора потомка требуется переставить точку исполнения кода на энтри-пойнт в соответствии с данными из эльф-хедера
    
- `free` 
- `readelf` - позволяет читать эльф-хедеры
- `editelf`
- `objdump` - утилита, позволяющая вывести информацию из секции

## 25. Процесс загрузки ELF в оперативную память. Entrypoint. 

https://en.wikipedia.org/wiki/Loader_(computing) https://en.wikipedia.org/wiki/Dynamic_linker

При запуске процесса в память загружается образ исполняемого файла, происходит динамическая линковка с разделяемыми библиотеками, размещение аргументов командной строки в памяти процесса, и затем происходит передача управления запускаемой программе: переход на ее entry point.

Entry point (см. https://en.wikipedia.org/wiki/Entry_point) — адрес первой инструкции процесса, с которой начинается ее исполнение. Он содержится в заголовке исполняемого файла в поле e_entry.

## 26. Процесс выделения памяти для ПО в ОС Linux.

**Shared memory** - сегмент памяти, информация из которого предоставляется одновременно сразу нескольким процессам (там находятся динамически-линкуемые библиотеки)  

В процессе выполнения `exec` выполняется реаллокация (отвязывание) shared-memory сегмента. После чего из elf-хедера выполняется чтение сегментов данных кода и т.д., которые переносятся в соответствующие сегменты процесса-потомка, а данные, оставшиеся там от форка родителя, удаляются. Стек и хип приводятся в первоначальный вид, после этого из шаред-мемори выполняется аллокация (присоединение) разделяемых блоков памяти. Если в шеред-мемори отсутствуют необходимые разделяемые эльф-объекты. По заключению подготовки виртуальной памяти, для процессора потомка требуется переставить точку исполнения кода на энтри-пойнт в соответствии с данными из эльф-хедера
    
- `free` 
- `readelf` - позволяет читать эльф-хедеры
- `editelf`
- `objdump` - утилита, позволяющая вывести информацию из секции

Каждый процесс исполняется в своем собственном адресном пространстве и за исключением случаев преднамеренного использования разделяемой памяти (shared memory) никоим образом не взаимодействует с памятью, выделенной другим процессам. С другой стороны, данные физически размещаются в оперативной памяти или файле подкачки. Это связано с тем, что виртуальные адресные пространства процессов и физическая память существуют независимо; в момент запроса на аллокацию памяти (например, при использовании библиотечной функции malloc) ядро выделяет процессу лишь адресное пространство; физическая память выделяется лишь по мере того, как в соответствующие адреса осуществляется запись данных.

Размер адресного пространства процесса ограничен лишь аппаратными возможностями процессора (на большинстве 64-битных процессоров это 2^48 байт); объем резидентной памяти ограничен доступными ресурсами оперативной памяти и разделов/файлов подкачки. Таким образом, может сложиться ситуация, когда для фактического выделения резидентной памяти при попытке записи ресурсов не хватит. Для того, чтобы гибко настраивать выделение адресного пространства процессам, существуют sysctl vm.overcommit_*.

https://www.kernel.org/doc/Documentation/vm/overcommit-accounting 

## 27. Работа с файлами и с памятью. Swap.

(Прим. ред. Нет уверенности, что это именно то, что требуется — формулировка очень расплывчатая).

В Linux есть три основных механизма работы с файлами: синхронный ввод/вывод; асинхронный ввод/вывод; файлы, отображенные в память (memory mapped files).

Синхронный ввод/вывдо осуществляется при помощи классических системных вызовов, обернутых в одноименные библиотечные функции — например, read, write, fsync. На время выполнения операции ввода/вывода приложение блокируется: возврат из системного вызова происходит только после окончания операции.

Асинхронный ввод/вывод позволяет выполнять запросы ввода/вывода, не блокируясь на время их выполнения: http://man7.org/linux/man-pages/man7/aio.7.html

Отображение файлов в память позволяет работать с содержимым файла как с данными в памяти процесса. https://en.wikipedia.org/wiki/Memory-mapped_file

Основные системные вызовы для работы с памятью — mmap и munmap - осуществляют ее аллокацию и освобождение соответственно; единица аллокации памяти для процесса — 1 страница; на архитектуре x86_64 размер страницы — 4 килобайта.

Пока среди уже выделенных страниц памяти достаточно места для аллокации запрашиваемых структур, никаких вызовов для аллокации дополнительной памяти не происходит; в противном случае выполняется системный вызов brk для изменения размера сегмента данных процесса, а затем вызывается mmap() для выделения памяти.

Для освобождения ставшей ненужной памяти используется unmap(). Например, следующий код на языке C

char * p = malloc(1024*1024);

<...>

free(p);

приведет к отправке следующих системных вызовов:

brk(NULL) = 0x55b58a6fa000

brk(0x55b58a71b000) = 0x55b58a71b000

mmap(NULL, 1052672, PROT_READ|PROT_WRITE, MAP_PRIVATE|MAP_ANONYMOUS, -1, 0) = 0x7f2af7dd1000

<...>

munmap(0x7f2af7dd1000, 1052672) = 0

Механизм подкачки (swap) используется для вытеснения редко используемых страниц памяти на диск, чтобы освободить место для более часто используемых данных — это могут быть как данные процессов, так и кеша (см. сл. вопрос). Политика вытеснения данных в swap определяется параметром swappiness: https://www.howtogeek.com/449691/what-is-swapiness-on-linux-and-how-to-change-it/ 